In [57]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Optional
import ipytest
ipytest.autoconfig()


In the workbook directory there is a file called `300x900c35.cti`. This is an spColumn CTI file and you can open it up in JupyterLab to view it. The default file format to save an spColumn file is the `.col` format, which is binary. However, you can also select to save a CTI file instead from the spColumn file save dialog to save your column as a text file.

You can read about how the `.cti` spColumn file format works here: https://structurepoint.org/spColumn-Online-Manual/spColumn/Appendix/spColumn_Text_Input_CTI_file_format.htm

### Note: For this workbook, it is expected that you will write tests for all of your functions even though it is not explicitly mentioned.

# Task A

Design a function called `read_cti_file` that takes filename (as a str) as the single parameter and returns the data in the file.

After a visual inspection of the `.cti` file, which file reading recipe, of the three we have used so far (in Lesson 02, Lesson 03, and Lesson 04), may be most advantageous? If you are not sure, try them out and see what the file data looks like after you read them.

## Task A Response

# Task B

Design a function that will loop through all of the data and convert any strings which are numbers into numbers.

The signature of this function will be as follows: `convert_strings_to_numbers(file_data: list) -> list`

Note: depending on how you have read your cti file data, your data may either be a string, a list of string, or a list of lists of strings. If it is a string, you should split your file data on the new line character `\n` so it is at least a list of lines.

#### Tips that _may_ be useful

1. There is a built-in function called `isinstance(obj, type)` that returns `True` if the `obj` is of the type given in `type`. e.g. 
```python
isinstance(item, list) # Either True or False
```

2. There is a technique in computer science known as _recursion_. When working with nested data (e.g. lists within lists) it is sometimes useful to use recursion. Often recursion is implemented as calling a function within that function's implementation (which is a thing you can do), e.g. 

```python
def print_nested_list_items(my_data: list) -> None:
    """
    Prints each individual item within a nested list structure regardless
    of how many levels deep they are.
    """
    for item in my_data:
        if isinstance(item, list):
            print_nested_list_items(item)
        else:
            print(item)
```

## Task B Response

# Task C

Design a **compound** data type to represent the broad structure of a `.cti` file. 

From a visual inspection of the CTI file, you may have noticed that the file is divided into sections, each section with its own header that looks something like this: `'[User Options]'`. For manipulating the CTI file, it would be useful to have the file data in a format that allows you to access the data in each section by the section name.

Now, there are lots of sections to a CTI file. By creating a `dict` data type, you can _dynamically_ create each of the heading names. With a `dataclass` data type, you would have to manually create each heading option as an attribute of your new class. It is ultimately up to you which direction you go, each with its "pros" and "cons".

Design a function that converts your file data into your new compound data type.

## Task C Response

# Task D

Now that you have your `.cti` data in your data type, you can manipulate it relatively easily. For example, you could change all of the factored loads by changing the values in the list of lists that represent the factored loads in your data type.

Design a function that takes two parameters, a `list[list[float]]` representing a factored load (Axial, Mx, My) and a `CTIFile` dictionary. **Your function will either return `None` -OR- it will return a new `CTIFile` dictionary (see "Important thing to consider", below).**

Note: From trial and error (and/or reading the CTI file documentation at: https://structurepoint.org/spColumn-Online-Manual/spColumn/Appendix/spColumn_Text_Input_CTI_file_format.htm), you will find that to update the factored loads (and have them be recognized by spColumn), you need to update three pieces of information.

1. In the `'[Factored Loads]'` section, you have to put in the new loads.
2. At the top of the `'[Factored Loads]'` section, there is an integer representing the count of the number of factored loads in the file.
3. In the `'[User Options]'` section, there is a single line of numbers in a list. These numbers have various meanings you can read about in the documentation. The number in the 18th position (if counting from 1) is an integer representing the count of the number of factored loads in the file. 

To update the factored loads, you must update **all three values**.

**Important thing to consider:** In a previous lesson, I explained that some Python data types are _mutable_ and some are _immutable_. Both the `dict` and the `list` are _mutable_ so when you update the CTIFile dictionary, it will change the dictionary that exists outside of the function, even if you do not `return` it. This means that if you try changing your factored loads with your function then you are effectively _over-writing_ your existing data with your new data and you cannot get it back without re-opening the original file again.

So, you have a decision that you can make: 
1. If you want to mutate the data in place then have your function return `None`. It will mutate your CTIFile in place and you do not need to return it.
2. If you want your function to mutate a _copy_ of your original CTIFile dictionary (therefore leaving your original CTIFile dict untouched) then your function returns the _new_ CTIFile dictionary.

Option 1 would look like this:
```python
my_cti_data # This is my CTIFile dictionary
update_factored_loads([[3000, 100, 100]], my_cti_data) # This function would return None so we do not assign the return value to a variable
print(my_cti_data["[Factored Loads]"]) # You would now see the updated load in the file data
```

Option 2 would look like this:
```python
my_cti_data # This is my CTIFile dictionary
my_updated_cti_data = update_factored_loads([[3000, 100, 100]], my_cti_data) # This function returns a copy with updated info so assign the copy to a new variable.

# These two would now show different data
print(my_updated_cti_data["[Factored Loads]"])
print(my_cti_data["[Factored Loads]"]) 
```

How do you make a copy in Python? You do so explicity by using `copy.deepcopy(obj)`.

e.g. 
```python
import copy

my_new_cti_data = copy.deepcopy(cti_data) # deepcopy copies the container (the dictionary) and every piece of data within it
```
If you are going with Option 2, perform your copy as the first line of your function and perform all of your operations on the copy. You _might_ also need to copy your `factored_loads` input, depending on how you assign your new loads to your CTIFile dictionary. Your function tests can help you troubleshoot this.

## Task D Response

# Task E

Now that you have written a function to update your factored loads, take a moment to appreciate what you now have: you have a program that can read any spColumn `.cti` file and allow you to easily update any of the data within it! Pretty neat, yeah?

Since your cti file data is stored in the `CTIFile` dictionary, you can now access any parts of the file you want and update them directly, e.g. the reinforcement, the f'c, even the shape of the section itself!

There is one final step: to write your updated data back to a _new_ `.cti` file.

Your task is this: design a function that takes two arguments, a new file name (`str`) and a `CTIFile` dictionary. The function writes the data within the dictionary to the disk with the new file name and returns `None`.

How to write a file to the disk?

## Task E Response

# Task F

Now that you have written these functions that can read, alter, and write `.cti` file data (and functions to test them), we are going to put them into proper _Python modules_.

1. Using JupyterLab, create a new Python file (click the blue "+" button at the top left and select "New Python File")
2. Name the file `cti_reader.py`
3. From the cells above, starting with the cell at the very top with the imports, copy your imports, functions, and data definition into the file. Put them in the following order: imports, data definition, functions
4. Create another new Python file and name the file `test_cti_reader.py`
5. At the top of the file write, `from cti_reader import *` and then copy all of your test functions into that file (you can omit the `ipytest.run()` parts)

**Restart your notebook.** Then, in the cells below, perform the following:

1. Type `import cti_reader` and run the cell. It should run without error.
2. Use your functions to read and alter the factored loads in the file `300x900c55.cti`. To access your functions from your module, you will type things like...

```python
import cti_reader

file_name = "300x900c55.cti"
file_data = cti_reader.read_cti_file(file_name)
converted_file_data = cti_reader.convert_strings_to_numbers(file_data)
# ... etc.
```
3. Write the updated file with a new file name
**4. Try opening your new file in spColumn and running the analysis**

# Submit Workbook 06! 🏌

So, how does it feel to have written a fully functional, usable Python library for manipulating spColumn files? 

I am hoping that you can get a sense of how you can build on to this library of yours with additional functions to change different parts of the `.cti` file. Updating other parts of the `.cti` file can get a little more involved than updating just the factored loads because you have to change even more data in multiple locations depending on the analysis mode you have selected in spColumn. However, if you want to spend a bit more effort on this, you can read the CTI file documentation further, try things out with trial-and-error, and you can also ask me.

BUT!!! You have written your first Python module!!! I would say that you now have The Fundamental Python Skills. These skills allow you to create data types to model real-life data, to write functions to create small programs, AND you know how to write unit tests to validate your functions. This is more than MANY get when they just start learning with online tutorials.

Next week, we are going to expand on your knowledge to try and cement it a bit more before we start venturing into the popular Pandas library for data analysis.

Please submit your Workbook 06 with the subject line, `Workbook 06 Submission` to `cferster@rjc.ca`.